### 鐵達尼號生存率練習
### https://github.com/fred17756/AI

In [1]:
import tensorflow as tf
import keras as kr

tf.__version__
kr.__version__
import pandas as pd
import numpy as np


Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
columns = train.columns
print(columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


## 觀察資料

In [3]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


#### Train Dataset 
#### 1. 需要將Age 補平均值
#### 2. 補Cabin的值為 if not null = 'NoCabin'
#### 3. Embarked的值為"s"

In [5]:
train['Embarked'] = train['Embarked'].fillna('S')
train['Cabin'] = train['Cabin'].apply(lambda x : str(x)[0] if not pd.isnull(x) else 'NoCabin')
train['Age'] = train['Age'].fillna(train['Age'].mean())

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [7]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


#### Test dataset
#### 1. 刪掉Age 補平均值
#### 2. 補Cabin的值為 'NoCabin' 讓他不為空值
#### 3. Fare 補 0 

In [9]:
test['Fare'] = test['Fare'].fillna(0.0)
test['Cabin'] = test['Cabin'].apply(lambda x : str(x)[0] if not pd.isnull(x) else 'NoCabin')
test['Age'] = test['Age'].fillna(train['Age'].mean())

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          418 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [11]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,30.154603,0.447368,0.392344,35.541956
std,120.810458,0.841838,12.636666,0.896760,0.981429,55.867684
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.471875
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


### 刪掉不要的變數
### 將名目資料轉為類別資料

In [12]:
columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

#### No PassengerId, Name , Ticket

In [13]:
train.drop(['PassengerId', 'Name','Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name','Ticket'], axis=1, inplace=True)

In [14]:
data = train.append(test)

In [15]:
data

,Age,Cabin,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived
0,22.000000,NoCabin,S,7.2500,0,3,male,1,0.0
1,38.000000,C,C,71.2833,0,1,female,1,1.0
2,26.000000,NoCabin,S,7.9250,0,3,female,0,1.0
3,35.000000,C,S,53.1000,0,1,female,1,1.0
4,35.000000,NoCabin,S,8.0500,0,3,male,0,0.0
5,29.699118,NoCabin,Q,8.4583,0,3,male,0,0.0
6,54.000000,E,S,51.8625,0,1,male,0,0.0
7,2.000000,NoCabin,S,21.0750,1,3,male,3,0.0
8,27.000000,NoCabin,S,11.1333,2,3,female,0,1.0
9,14.000000,NoCabin,C,30.0708,0,2,female,1,1.0


In [16]:
data['Sex'] = data['Sex'].astype('category').cat.codes
data['Embarked'] = data['Embarked'].astype('category').cat.codes
data['Pclass'] = data['Pclass'].astype('category').cat.codes
data['Cabin'] = data['Cabin'].astype('category').cat.codes

In [17]:
train_data = data[pd.notnull(data['Survived'])]
test_data = data[pd.isnull(data['Survived'])]
train_data.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp',
       'Survived'],
      dtype='object')

## Training Model Start

In [18]:
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC

C:\Users\skbank\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
#資料準備
train_data_X = train_data[['Age', 'Cabin', 'Embarked', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp']]
train_data_Y = train_data['Survived']
test_data_X = test_data[['Age', 'Cabin', 'Embarked', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp']]
#分割資料
train_X, test_X, train_y, test_y = train_test_split(train_data_X,train_data_Y, test_size=0.3)
#模型
svm = SVC(kernel = 'linear',probability = True)
svm.fit(train_X,train_y)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [33]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

p = svm.predict(test_X)
svm.score(test_X,test_y)

a = accuracy_score(test_y,p)
print(a)
confusion_matrix(test_y, p)

0.817164179104


array([[137,  14],
       [ 35,  82]], dtype=int64)

#### 要將這個結果輸出

In [ ]:
#模型
svm = SVC(kernel = 'linear',probability = True)
svm.fit(train_data_X,train_data_Y)
svm.predict(test_data_X)